In [2]:
import pandas as pd

changes made 5/30: (feel free to look at github history if you want to revert some stuff)
- loaded in court_list and processed judges outside of function
- loaded in detail_codes for eventual mapping 
- used map instead of merge to replace court id with court name outside of function - looks cleaner
- took out all the stuff with the judges dataframe since it's already been moved over to the script
- creating a function for mapping codes with actual meanings
    - started the probation type column b/c "No Probation" and "NP" was my mistake when scraping

In [ ]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
codes = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/detail_codes.csv")
#dictionary for replacing the court id with the court name
court_code = dict(courts[["Court ID", "Court Name"]].values)

In [ ]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    #standardized between courts
    df["Probation Type"][df["Probation Type"] == "No Probation"] = "NP"
    #gets probation codes from list

    return df

I'm thinking it might be better to load in the stuff outside the function when generalizing if we're eventually going to concatenate them all together

In [ ]:
def case_matcher(cases):
    """
    cases: a dataframe of cases for 1 specifc court
    """
#     case_url="/Users/hinaljajal/Downloads/{}.csv".format(court+"_cases")
#     cases=pd.read_csv(case_url)

    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    #checks if there are are different judges with the same initials
    judges1 = judges.drop_duplicates(subset=['Judge Full Name'])
    boolean = judges1.duplicated(subset=['Judge']).any()
    
    #If there are duplicates
    if (boolean):
        judges['Year']=judges['Year'].astype(int)
        cases['Year']=cases['Year'].astype(int)
        #Merges the columns by the judge and the hearing year
        cases = cases.merge(judges, how="left",on = ["Judge", "Year"])
    
    #Saves the dataframe to a csv
    file_name = '/Users/hinaljajal/Downloads/{}.csv'.format(court)
    cases.to_csv(file_name, index = False)
    
    return cases

In [ ]:
separate_court_data = []
for court in court_list["Court ID"]:
    data = pd.read_csv(f"path/to/files/{court}.csv")
    data = map_values(data)
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)

In [ ]:
#put in database?